In [ ]:
import os
import re # importing regular expression that helps to find other strings or set of strings
import win32com.client
import pandas as pd
import numpy as np
import math
from win32com.client import constants
import datetime
import shutil
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.listitems.caml.caml_query import CamlQuery
from office365.runtime.auth.user_credential import UserCredential
import glob



# main class of visio file generate
class main():
    try:
        # initial function to initialise all default used variables and values we need in visio creation
        def __init__(self, excelPath, visioPath, swimlaneTitle, pdfDestination, visioDestination):
            # getting the access of invisible viso app access
            self.appVisio = win32com.client.Dispatch("Visio.InvisibleApp")
            self.appVisio.Visible = 0  # set visibility to 0 do dont show popup of visio software
            self.visSectionObject = 1  # default 0
            self.visRowPage = 10  # rows in a page
            self.visPageWidth = 10  # default 0
            self.visPageHeight = 1
            self.visCharacterDblUnderline = 8
            self.visCharacterColor = 1
            self.visSectionCharacter = 3
            self.visioPath = visioPath  # default visio path
            self.excelPath = str(excelPath)  # excel path define
            self.visioFileName = str(visioDestination)  # visio file define to save visio with the file name to destination folder
            self.sheetName = 'Description'  # sheet name of excel to read the description need in visio title

            self.pdfDestination = pdfDestination  # pdf destination defined
            self.swimlaneTitle = swimlaneTitle  # swim lane title defined

            self.doc = self.appVisio.Documents.Add(visioPath)  # assigning the default visio app to doc veriable
            self.pagObj = self.doc.Pages.Item(1)

            self.stnObj_xFunc = self.appVisio.Documents("XFUNC_U.vssx")  # this is used to access the function objects
            self.stnObj_Basic = self.appVisio.Documents("BASFLO_U.vssx")  # this is used to access the basic object

            self.readFile()  # raed excel file function called

        # defining function here
        def defineFunction(self, functionsName):
            # x and y have the point where to draw the diagram
            x = -13.2
            y = -24.5

            # this will store the function name array
            self.functionNameArr = []
            # for loop to add function names to function name array
            for nameFun in functionsName:
                # appendng the function name to function name array with self. string and also replacing the white space to underscore
                self.functionNameArr.append("self." + re.sub("[^a-zA-Z0-9]", "_", nameFun.replace("\n", "")))

            function = self.stnObj_xFunc.Masters("Swimlane")  # swimlane is defined to function

            # by using exec (execute) function we are defining function name dynamically
            exec("{} = self.pagObj.Drop(function, x, y)".format(self.functionNameArr[0]))
            # assign the text to function name
            eval(self.functionNameArr[0]).Text = str(functionsName[0])

            # function.Title = str(functionsName[0])
            eval(self.functionNameArr[0]).Cells("Height").FormulaU = "2 in"
            functionsName.pop(0)
            self.functionNameArr.pop(0)
            shp = self.pagObj.Shapes.Item("CFF Container")
            shapeSwim = None
            members = shp.ContainerProperties.GetMemberShapes(4) #IDs of all the shapes in the flow chart excluding the shapes which contails additional details

            # shape = self.pagObj.DrawRectangle(0.375, 16.0354, 2, 2)

            readFile = pd.ExcelFile(self.excelPath)  # reading excel file
            df = readFile.parse(self.sheetName)  # parsing the sheet name "description" to read the description sheet data

            val = np.concatenate(df.iloc[:, 0:4].values)  # description values getting from excel
            val[9]= pd.to_datetime(val[9]).tz_localize(None)

            dtMod = val[9].isoformat().split('T')[0]  # date format changing
            description = '''{}{}\n{}{}\n{}{}'''.format(df.keys()[0], df.keys()[1], val[0], val[1], val[4], val[5])  # this is description 1 data from description sheet
            description2 = '''{}{}\n{}{}\n{}{}'''.format(df.keys()[2], df.keys()[3], val[2], val[3], val[6], val[7])  # this is description 2 data from description sheet
            description3 = '''{}{}\n{}{}\n{}{}'''.format(val[8], dtMod, val[10], val[11], val[12], val[13], val[16],  # this is desxription 3 Data from description sheet
                                                         val[17])

            # drawing description 1
            title = self.stnObj_Basic.Masters("Process")
            title = self.pagObj.Drop(title, 4.3531, 10.71)
            title.Cells("Width").FormulaU = "8.034 in"
            title.Cells("Height").FormulaU = "0.51 in"
            title.Cells("Fillforegnd").FormulaU = "RGB(79, 136, 187)"
            title.text = description

            # drawing description 2
            title = self.stnObj_Basic.Masters("Process")
            title = self.pagObj.Drop(title, 12.3532, 10.71)
            title.Cells("Width").FormulaU = "8.034 in"
            title.Cells("Height").FormulaU = "0.51 in"
            title.Cells("Fillforegnd").FormulaU = "RGB(79, 136, 187)"
            title.text = description2

            # drawing description 3
            title = self.stnObj_Basic.Masters("Process")
            title = self.pagObj.Drop(title, 8.3531, 10.2)
            title.Cells("Width").FormulaU = "16.0354 in"
            title.Cells("Height").FormulaU = "0.51 in"
            title.Cells("Fillforegnd").FormulaU = "RGB(79, 136, 187)"
            title.text = description3

            # title.Cells("Align").FormulaU = ""

            otherDetailAtPhaseLocationProcessShape = self.stnObj_Basic.Masters("Process")
            otherDetailAtPhaseLocation = self.pagObj.Drop(otherDetailAtPhaseLocationProcessShape, 8.3531, 9.8183)
            otherDetailAtPhaseLocation.Cells("Width").FormulaU = "16.0354 in"
            otherDetailAtPhaseLocation.Cells("Height").FormulaU = "0.25 in"
            otherDetailAtPhaseLocation.Cells("Fillforegnd").FormulaU = "RGB(79, 136, 187)"
            otherDetailAtPhaseLocation.Text = self.swimlaneTitle

            for member in members:
                if (self.pagObj.Shapes.ItemFromID(member)).Name == "Swimlane List":
                    shapeSwim = self.pagObj.Shapes.ItemFromID(member)
            for index, f in enumerate(functionsName):
                exec("{} = self.appVisio.ActivePage.DropIntoList(function, shapeSwim, 4)".format(
                    self.functionNameArr[index]))
                eval(self.functionNameArr[index]).text = str(f)
                eval(self.functionNameArr[index]).Cells("Height").FormulaU = "2 in"
            self.createChart()

        def createChart(self):
            # x and y is for chart diagram
            x = 0
            y = 0
            position = 0
            # assigning the required fields data to variables
            processId, processDicription, shapeType, function, controlId, endCustomer, additionalDetails = self.process[
                                                                                                               "Process Step ID"].to_numpy(), \
                                                                                                           self.process[
                                                                                                               "Process Step Description"].to_numpy(), \
                                                                                                           self.process[
                                                                                                               "Shape Type"].to_numpy(), \
                                                                                                           self.process[
                                                                                                               "Function"].to_numpy(), \
                                                                                                           self.process[
                                                                                                               "Control ID"].to_numpy(), \
                                                                                                           self.process[
                                                                                                               "End Customer"].to_numpy(), \
                                                                                                           self.process[
                                                                                                               "Additional Details"].to_numpy()
            shapes = pd.unique(shapeType)  # creating unique shape names
            funcArr = pd.unique(function)  # to define unique functions name
            # for loop to work on shape array to define them
            for index, shape in enumerate(shapes):
                if "Start" in shape or "End" in shape:
                    # start and end will define here if shape is start or end
                    exec("startEnd = self.stnObj_Basic.Masters('Start/End')")
                else:
                    # if shape is not start and end then it will defined from here
                    exec("{} = self.stnObj_Basic.Masters('{}')".format(re.sub("[^a-zA-Z0-9]", "", shape),
                                                                       re.sub("[^a-zA-Z0-9]", "", shape)))

            # this for loop is define the function from funcArr
            for index, f in enumerate(funcArr):
                exec("{}_len = len(self.process[self.process['Function'] == '{}'])".format(
                    re.sub("[^a-zA-Z0-9]", "_", f.replace("\n", "")), f.replace("\n", "")))
                swimlaneHight = eval(re.sub("[^a-zA-Z0-9]", "_", f.replace("\n", "")) + "_len") // 6
                if swimlaneHight >= 1:
                    if swimlaneHight < 2:
                        manageLen = -1
                    else:
                        manageLen = 0
                    eval("self." + re.sub("[^a-zA-Z0-9]", "_", f)).Cells("Height").FormulaU = "{} in".format(
                        3.5 + manageLen + swimlaneHight)

            for index, f in enumerate(funcArr):
                exec("{} = []".format(re.sub("[^a-zA-Z0-9]", "_", f.replace("\n", ""))))

            # this for loop used to define process id variable and define the processes in visio
            for index, id in enumerate(processId):
                funcData = re.sub("[^a-zA-Z0-9]", "_", function[index].replace("\n", ""))
                distanceArr = eval(funcData)
                pos = len(distanceArr)
                setY = eval(funcData + "_len") // 6
                if setY != 0:
                    exec("{}_len -= 1".format(funcData))
                    y = setY + setY / 2
                else:
                    y = 0

                if pos == 0:
                    position = pos + 1.5
                else:
                    if pos % 6 != 0:
                        position = distanceArr[pos - 1] + 2.4
                    else:
                        position = 1.5
                # condition for start and end shape type to set color and text
                if "Start" in shapeType[index] or "End" in shapeType[index]:
                    exec("{} = {}.Drop(startEnd, x + {}, y + 1)".format(re.sub("[^a-zA-Z0-9]", "", id),
                                                                        "self." + funcData, position))
                    eval("{}".format(re.sub("[^a-zA-Z0-9]", "", id))).Text = str(processDicription[index])
                    if shapeType[index] == "Start":
                        eval("{}".format(re.sub("[^a-zA-Z0-9]", "", id))).Cells(
                            "Fillforegnd").FormulaU = "RGB(0, 128, 0)"
                    else:
                        eval("{}".format(re.sub("[^a-zA-Z0-9]", "", id))).Cells(
                            "Fillforegnd").FormulaU = "RGB(255, 0, 0)"
                    distanceArr.append(position)
                else:
                    # here we set the other diagram except the start and end
                    exec("{} = {}.Drop({}, x + {} , y + 1)".format(re.sub("[^a-zA-Z0-9]", "", id),
                                                                   "self." + funcData, shapeType[index],
                                                                   position))
                    if shapeType[index] == "Database":
                        eval("{}".format(re.sub("[^a-zA-Z0-9]", "", id))).Cells("Angle").FormulaU = "90 deg"
                    eval("{}".format(re.sub("[^a-zA-Z0-9]", "", id))).Text = str(processDicription[index])
                    
                    # checking the control id is null or not
                    if str(controlId[index]) != "nan":
                        val = 0.6
                        # for loop to control id text if there is line break
                        for i, controlIdText in enumerate(str(controlId[index].replace("\n", ",")).split(",")):
                            if i >= 1:
                                val += 0.3
                            process = eval("Process")
                            if int(y) == 0:
                                pinY = 0.2
                            else:
                                pinY = "0." + str(int(y) + int(y))
                            controlIdDefine = eval("{}".format(
                                "self." + funcData)).Drop(
                                process, x + position, y + 1)
                            
                            controlIdDefine.Cells("Width").FormulaU = "Sheet.12!Width*0.0{}93 in".format(
                                len(re.sub("[^a-zA-Z0-9]", "", controlIdText)) // 2)
                            controlIdDefine.Cells("Height").FormulaU = "Sheet.1!Height*0.0937 in"
                            controlIdDefine.Cells("LocPinY").FormulaU = "Height+{} in".format(val)
                            controlIdDefine.Cells("Fillforegnd").FormulaU = "RGB(255, 255, 153)"
                            controlIdDefine.CellsSRC(self.visSectionCharacter, 0,
                                                     self.visCharacterColor).FormulaU = "RGB(0,0,0)"
                            controlIdDefine.Text = controlIdText
                            # controlId diagram rotating to 90 degree 
                            controlIdDefine.Cells("Angle").FormulaU = "90 deg"
                            
                    # checking is end customer data is null in excel
                    if str(endCustomer[index]) != "nan":
                        process = eval("Process")
                        controlIdDefine = eval("{}".format(
                            "self." + funcData)).Drop(
                            process, x + position, y + 1)
                        controlIdDefine.Cells("Width").FormulaU = "0.3409 in"
                        controlIdDefine.Cells("Height").FormulaU = "0.2557 in"
                        controlIdDefine.Cells("LocPinX").FormulaU = "width+0.1 in"
                        controlIdDefine.Cells("LocPinY").FormulaU = "height-0.7 in"
                        controlIdDefine.Cells("Fillforegnd").FormulaU = "RGB(0, 236, 106)"
                        controlIdDefine.CellsSRC(self.visSectionCharacter, 0,
                                                 self.visCharacterColor).FormulaU = "RGB(0,0,0)"
                        controlIdDefine.Text = str(endCustomer[index])
                        
                    # checking is additional details are null or not
                    if str(additionalDetails[index]) != "nan":
                        if int(y) == 0:
                            pinY = 1.5
                        else:
                            pinY = 1.5

                        document = self.stnObj_Basic.Masters("Document")
                        controlIdDefine = eval("{}".format(
                            "self." + funcData)).Drop(
                            document, x + position, y + 1)
                        controlIdDefine.Cells("Width").FormulaU = "User.DefaultWidth"
                        controlIdDefine.Cells("Height").FormulaU = "0.5 in"
                        controlIdDefine.Cells("LocPiny").FormulaU = "Height+0.6 in".format(pinY)
                        controlIdDefine.Text = str(additionalDetails[index])
                    # appending the last position of diagram
                    distanceArr.append(position)

            nextStep = list(self.process["Next Step ID"].fillna("nan"))  # storing the next step id list from excel
            
            # for loop to connect (glue) the two diagrams by arrow 
            for index, id in enumerate(processId):
                if len(nextStep) == index:
                    pass
                else:
                    # checking is nextStep value is null in excel or not
                    if nextStep[index] != "nan":
                        # for loop to next step array
                        for stepIndex, step in enumerate(nextStep[index].split(",")):
                            # removing white space and any symbol exist in text
                            steps = re.sub("[^a-zA-Z0-9]", "", step)
                            ids = re.sub("[^a-zA-Z0-9]", "", id)
                            # connector tool defined here
                            connectorMaster = self.appVisio.Application.ConnectorToolDataObject
                            exec("{}_connector_{} = self.pagObj.Drop(connectorMaster, 0, 0)".format(ids, steps))
                            connectorLable = self.process["Connector Label"][index]
                            # checking is connector label is null or not
                            if str(connectorLable) != "nan":
                                if len(connectorLable.split(",")) > 1:
                                    self.text = connectorLable.split(",")[stepIndex]
                                elif len(connectorLable.split(",")) == 1:
                                    self.text = connectorLable.split(",")[0]
                                eval("{}_connector_{}".format(ids, steps)).Text = self.text
                            # pointing the beginning x point to start the connector 
                            eval("{}_connector_{}".format(ids, steps)).Cells("BeginX").GlueTo(eval(ids).Cells("PinX"))
                            # pointing the end x point to start the connector                            
                            eval("{}_connector_{}".format(ids, steps)).Cells("EndX").GlueTo(
                                eval("{}".format(steps)).Cells("PinY"))
                    else:
                        pass
            # checking is there is pdf destination to save the pdf format of visio
            if self.pdfDestination != "":
                self.time = datetime.datetime.now().strftime("%d-%m-%Y %I-%M-%S%p")
                pdfFilePath = r"{}\Visio_Pdf_{}.pdf".format(self.pdfDestination, self.time)
                self.doc.ExportAsFixedFormat(1, pdfFilePath, 1, 0)
                os.startfile("{}".format(pdfFilePath))
                
            # here we are saving the generated visio file 
            self.doc.SaveAs(self.visioFileName)
            self.doc.Close()  # closing the default visio file
            self.appVisio.Quit()  # closing the visio app
            
        # here we read the excel file
        def readFile(self):
            # reading the excel file using pandas
            self.data = pd.ExcelFile(self.excelPath)
            self.process = self.data.parse(0) #taking out the Process Map sheet name from the excel file
            self.function = self.process["Function"].to_numpy()
            self.uniqueFunctions = list(pd.unique(self.function)) #this gives a list of unique value of function from the excel file sheet name Process Map
            self.defineFunction(self.uniqueFunctions)  # define function called to generate the visio diagram

    except Exception as e:
        # here you get error if there occur in visio generate
        print(e)
        print('Visio generate error-->', e)


class genrateVisio():
    def __init__(self, sourceFolder, destinationFolder):
        # at initial level checking is vsdx file exist in source folder as name of excel name
        self.check_vsdx_file_for_same_excel_name(sourceFolder=sourceFolder, destinationFolder=destinationFolder)

    # check vsdx file for same excel name is exist or not (ex. source/dir: demo_unique_name.xlsx and in destination/dir: demo_unique_name.vsdx)
    def check_vsdx_file_for_same_excel_name(self, sourceFolder, destinationFolder):
        # listing files from source folder in array
        sourceFolderFiles = os.listdir(sourceFolder) #gives the list of files present in this directory
        # listing files from destination folder in array
        destinationFolderFiles = os.listdir(destinationFolder)

        # this condition checking is there are files exist in source folder or not
        if len(sourceFolderFiles) > 0:
            # loop on source folder files
            for sourceFile in sourceFolderFiles:
                # replacing the source excel file to vsdx file extension (ex. demo_unique_name.xlsx => demo_unique_name.vsdx)
                destinationCheckVsdxFile = sourceFile.replace('.xlsx', '.vsdx') \
                    .replace('.xls', '.vsdx') \
                    .replace('.csv', '.vsdx') \
                    .replace('.xlsm', '.vsdx')
                # this condition check is vsdx file name is exist in destination folder files array if not the genrate visio diagram
                if destinationCheckVsdxFile not in destinationFolderFiles:
                    # getting visio default file path (NOTE: default_visio_file.vstx file is default file for the process)
                    self.visioPath = os.getcwd() + "\default_visio_file.vstx"
                    # excel file path defining to excelPath veriable
                    excelPath = sourceFolder + sourceFile
                    # this print indicates the visio generate process in progress
                    print(f'Processing "{sourceFile}"...')
                    # defining the visio file path in server
                    visioDestination = destinationFolder.replace('/', '\\') + destinationCheckVsdxFile
                    # calling the main visio generate class
                    main(visioPath=self.visioPath, excelPath=excelPath, swimlaneTitle="Enter Title", pdfDestination="",
                         visioDestination=visioDestination)
                    # this print indicate processing on the file is done
                    print(f'Processing "{sourceFile}" is done.')
                else:
                    # this print indicate file is already exist and if you are given the different then check your file name and upload excel again with unique name
                    print(
                        f'Process of "{sourceFile}" is already done.\n(NOTE: Please check destination folder with same excel name.)')


if __name__ == "__main__": #through this , firstly it will be executed first in this code
    # source folders directory to get the excel fiels
    sourceFolder = 'D:/HCFM Project _ Code/Master/Source_Folder/'
    # destination folder to check the excel file name with vsdx extension is exist or not and if not exist then save file to this dir
    destinationFolder = 'D:/HCFM Project _ Code/Master/Dest_Folder/'
    genrateVisio(sourceFolder, destinationFolder)  # generate visio function

#### Code to upload vsdx file to sharepoint sub folder ####
dest_path = 'D:/HCFM Project _ Code/Master/Dest_Folder/'

baseurl = 'https://my.metlife.com'
basesite = '/teams/ia/Powerappsdatabase/' # every share point has a home.
siteurl = baseurl + basesite 


ctx = ClientContext(siteurl).with_credentials(UserCredential("akshi.garg@metlife.com", "March8892"))

vsdxCounter = len(glob.glob1(dest_path,"*.vsdx")) #glob.glob1 with the help of wildcard character gives the list of all the files with that extension

for i in range((vsdxCounter)+1):
    for files in os.listdir(dest_path):
        if files.endswith('.vsdx'):
            f_name1 = dest_path + files
            remotepath = "Shared Documents/HCFM/"+ files
            with open(f_name1, 'rb') as content_file: #open the file in binary format for reading rb means that here
                file_content = content_file.read()

            dir, name = os.path.split(remotepath) #this breaks remotepath in its head and tail dir and name respectively
            file = ctx.web.get_folder_by_server_relative_url(dir).upload_file(name, file_content).execute_query()

print("Process Finished Successfully")